<a href="https://colab.research.google.com/github/cabralchege/The-Geothermal-Champions-League-A-Global-Energy-Analysis/blob/main/geothermal_leaders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Geothermal Energy Analysis

In [20]:
# Libraries
import pandas as pd
import pandas_gbq
from google.colab import auth
from google.cloud import bigquery
from google.colab import userdata

In [21]:
# Authenticate
auth.authenticate_user()


In [22]:
# Fetch project ids
try:
    PROJECT_ID = userdata.get('GCP_PROJECT_ID')
    DATASET_ID = userdata.get('BQ_DATASET_ID')
    RAW_TABLE_NAME = 'eia_raw_manual'
    CLEAN_TABLE_NAME = 'Cleaned_Geothermal_Data'
except Exception as e:
    print("Project id's error")
    raise e

## Extracting data from Big Query

In [23]:
# Extract
geothermal = bigquery.Client(project=PROJECT_ID)

# query
query = f"""
    SELECT * FROM `{PROJECT_ID}.{DATASET_ID}.{RAW_TABLE_NAME}`
"""

## Loading the dataset

In [24]:
# Load data
df = geothermal.query(query).to_dataframe()
print(df.shape)

(229, 46)


## Data Cleaning
- The dataset had spaces befor country names, null values were dashes and some had the s sign.

In [25]:
# Apply correct headers
years = [str(y) for y in range(1980, 2024)]
correct_columns = ['API', 'Country'] + years

# Add column names to dataframe
df.columns = correct_columns[:len(df.columns)]

# Remove duplicate headers if added as part of data
if df.iloc[0]['Country'] == 'Country':
    df = df.iloc[1:].reset_index(drop=True)

df.head()

,API,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,INTL.35-12-FSM-BKWH.A,Micronesia,None,None,None,None,None,None,None,None,...,None,None,None,None,NA,NA,NA,NA,NA,NA
1,INTL.35-12-TUV-BKWH.A,Tuvalu,None,None,None,None,None,None,None,None,...,None,None,None,None,NA,NA,NA,NA,NA,NA
2,INTL.35-12-XKS-BKWH.A,Kosovo,--,--,--,--,--,--,--,--,...,0,0,0,0,0,0,0,0,0,0
3,INTL.35-12-MNE-BKWH.A,Montenegro,--,--,--,--,--,--,--,--,...,0,0,0,0,0,0,0,0,0,0
4,INTL.35-12-SRB-BKWH.A,Serbia,--,--,--,--,--,--,--,--,...,0,0,0,0,0,0,0,0,0,0


In [32]:
# Clean country names
df['Country'] = df['Country'].str.strip()

# Remove the api code column
if 'API' in df.columns:
    df = df.drop(columns=['API'])

In [33]:
# Change the dataset to long format
df_long = df.melt(id_vars=['Country'], var_name='Year', value_name='Generation_TWh')

# Clean numbers
df_long['Generation_TWh'] = pd.to_numeric(df_long['Generation_TWh'], errors='coerce')
df_long['Generation_TWh'] = df_long['Generation_TWh'].fillna(0)

In [34]:
# Year is a number
df_long['Year'] = df_long['Year'].astype(int)

# Filter
df_clean = df_long [df_long['Year'] >= 2000].copy()

In [35]:
# Rank
latest_year = df_clean['Year'].max()
latest_data = df_clean[df_clean['Year'] == latest_year].copy()
latest_data['Rank'] = latest_data['Generation_TWh'].rank(ascending=False, method='min')

# Merge rank back
df_clean = df_clean.merge(latest_data[['Country', 'Rank']], on='Country', how='left')

In [36]:
# Add Story Group
def assign_group(country):
    if country in ['United States', 'Indonesia', 'Philippines']:
        return 'The Old Guard'
    elif country in ['Turkiye', 'Kenya', 'New Zealand']:
        return 'The New Challengers'
    elif country in ['Iceland', 'Mexico', 'Italy', 'Japan']:
        return 'Other Leaders'
    else:
        return 'Rest of the World'

df_clean['Story_Group'] = df_clean['Country'].apply(assign_group)

In [37]:

# Save file
output_filename = "Cleaned_Geothermal_Data.csv"
df_clean.to_csv(output_filename, index=False)

print(f" Saved {len(df_clean)} rows to {output_filename}")

 Saved 5496 rows to Cleaned_Geothermal_Data.csv


In [39]:
# Load clean data to big query
df_clean.to_gbq(f"{DATASET_ID}.{CLEAN_TABLE_NAME}",
    project_id = PROJECT_ID,
    if_exists = "replace",
)

print("Data warehouse updated")

/tmp/ipython-input-2976282636.py:2: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  df_clean.to_gbq(f"{DATASET_ID}.{CLEAN_TABLE_NAME}",
100%|██████████| 1/1 [00:00<00:00, 6689.48it/s]

Data warehouse updated
